In [37]:
import yfinance as yf
import numpy as np
import pandas as pd

In [38]:
df = pd.read_json('/Users/blakeuribe/Desktop/portfolio_py/data/raw/company_tickers.json')


df = df.T
df = df.drop(columns=['cik_str', 'title'])


In [39]:

start_date = "2023-12-20"
# Download T-Bill data once
tbill = yf.download('^IRX', start=start_date, end='2024-12-20', auto_adjust=True)['Close']
tbill = tbill / 100 / 360  # Convert to daily rate

[*********************100%***********************]  1 of 1 completed


In [40]:
# Define function to download and calculate Sharpe for a single ticker

def calculate_sharpe_ratio(ticker, tbill=tbill, start_date=start_date, end_date='2024-12-20'):
    # has about +-0.15 margin of error
    stock_data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=True)['Close']
    # Calculate daily returns for the ticker only, without trying to assign multiple columns
    stock_data[f'{ticker}_daily_ret'] = stock_data.pct_change()  # daily returns
    stock_data.dropna(inplace=True)  # drop NaNs
    
    # print('\n')
    # print(f'collected data, and pct change on {ticker}')
    
    tbill = tbill.reindex(stock_data.index, method='ffill')  # Align risk-free rates to stock dates

    # Calculate excess returns (subtract T-Bill rate)
    excess_returns = stock_data[f'{ticker}_daily_ret'] - tbill['^IRX']
    excess_returns_std = excess_returns.std()
    average_excess_daily_ret = excess_returns.mean()

    # print(f'calculating excess ret on {ticker}')
   
    # Calculate Sharpe ratio
    daily_sharpe_ratio = average_excess_daily_ret / excess_returns_std
    # Using 360 days results in a closer sharpe compared to website like portfolioslab.com
    annualized_sharpe = daily_sharpe_ratio * np.sqrt(360)

    # print(f'calculating annulize sharpe on {ticker}')
    # print(f'Type for {ticker} sharpe variable is: {type(annualized_sharpe)}')
    # print('\n')

    return annualized_sharpe

In [41]:

# Initialize results DataFrame
results_df = pd.DataFrame(columns=['ticker', 'sharpe_ratio'])

# Process tickers and store results
for ticker in df['ticker'][0:5000]:
    try:
        sharpe = calculate_sharpe_ratio(ticker, tbill)
        # Append new row to results DataFrame
        new_row = pd.DataFrame({
            'ticker': [ticker],
            'sharpe_ratio': [sharpe]
        })
        results_df = pd.concat([results_df, new_row], ignore_index=True)
    except Exception as e:
        print(f'Error processing {ticker}: {str(e)}')

# Sort by Sharpe ratio (optional)
results_df = results_df.sort_values('sharpe_ratio', ascending=False)

[*********************100%***********************]  1 of 1 completed
/var/folders/cd/2yq8cm193ln9jmnq_z6wtxnw0000gn/T/ipykernel_33079/2618665066.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_row], ignore_index=True)
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[***

In [42]:
print(f'Pct of Nan value is: {(results_df.isna().sum().sum()/len(results_df)) *100}%')
results_df

Pct of Nan value is: 15.7%


,ticker,sharpe_ratio
4595,LNKS,8.709793
4346,IZTC,8.098648
3475,JUNS,7.953146
4095,AVR,7.717997
4074,CGTL,7.352526
...,...,...
4769,ITMSF,NaN
4770,NIXX,NaN
4771,OPI,NaN
4772,PHUN,NaN


In [43]:
results_df.to_csv('/Users/blakeuribe/Desktop/portfolio_py/data/clean/sharpe_ratios.csv', index=False)